In [7]:
from math import nan
import numpy as np
import pandas as pd

from scipy.optimize import minimize, LinearConstraint

from reopt_interface import setup, create_post, run_reopt, parse_dispatch_series, \
    calc_retail_electric_cost, plotly_stacked, calc_monthly_peaks2, calc_mae2, build_price_vectors

# to silence warnigns
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

pd.options.plotting.backend='plotly'

with open('api_polimi.key', 'r') as f:
    API_KEY = f.read()
    
cfg,tariff,load,forecasts = setup('jpl.yaml')

In [5]:
solar = pd.read_csv('data/load/Jail_load_solar_1min_2021-4-21_2022-4-19_profilized.csv',
                    index_col=0,
                    parse_dates=True,
                    comment='#',)[['Solar [kW]']].clip(lower=0).resample('1h').mean()
solar

,Solar [kW]
Datetime CST,
2019-01-01 00:00:00,0.0
2019-01-01 01:00:00,0.0
2019-01-01 02:00:00,0.0
2019-01-01 03:00:00,0.0
2019-01-01 04:00:00,0.0
...,...
2019-12-31 19:00:00,0.0
2019-12-31 20:00:00,0.0
2019-12-31 21:00:00,0.0


In [20]:
forecasts = solar.copy(deep=True)#pd.read_csv('data/load/2019_JPL_v4.csv',index_col=0,parse_dates=True)

maxload = forecasts[cfg.solar_col].max()

forecasts['Perfect'] = forecasts[cfg.solar_col]
print('Perfect',calc_mae2(forecasts['Perfect'],forecasts['Perfect'],normalize=True))

#forecasts['Persist7d'] = forecasts[cfg.load_persist_col]
#print('Persist7d',calc_mae2(forecasts['Perfect'],forecasts['Persist7d'],normalize=True))

# nmae 5%

forecasts['ConstantDn'] = forecasts['Perfect'] - 0.1 * forecasts['Perfect'].max()
print('ConstantDn',calc_mae2(forecasts['Perfect'],forecasts['ConstantDn'],normalize=True))

forecasts['ConstantUp'] = forecasts['Perfect'] + 0.1 * forecasts['Perfect'].max()
print('ConstantUp',calc_mae2(forecasts['Perfect'],forecasts['ConstantUp'],normalize=True))


forecasts['ScaleLinDn'] = forecasts['Perfect'] - forecasts['Perfect'] * 0.67179736
forecasts['ScaleLinUp'] = forecasts['Perfect'] + forecasts['Perfect'] * 0.67179736
print('ScaleLinDn',calc_mae2(forecasts['Perfect'],forecasts['ScaleLinDn'],normalize=True))
print('ScaleLinUp',calc_mae2(forecasts['Perfect'],forecasts['ScaleLinUp'],normalize=True))

forecasts['ScaleQuadDn'] = forecasts['Perfect'] - forecasts['Perfect']**2 * 1.14869162/maxload
forecasts['ScaleQuadUp'] = forecasts['Perfect'] + forecasts['Perfect']**2 * 1.14869162/maxload
print('ScaleQuadDn',calc_mae2(forecasts['Perfect'],forecasts['ScaleQuadDn'],normalize=True))
print('ScaleQuadUp',calc_mae2(forecasts['Perfect'],forecasts['ScaleQuadUp'],normalize=True))

# tou_levels = np.array([tariff['energyweekdayschedule'][M-1][h] for M,h in zip(forecasts.index.month,forecasts.index.hour)])
# forecasts['TouScaleDn'] = forecasts.Perfect.values - maxload*0.0333 * tou_levels
# forecasts['TouScaleUp'] = forecasts.Perfect.values + maxload*0.0333 * tou_levels
# print('TouScaleDn',calc_mae2(forecasts['Perfect'],forecasts['TouScaleDn'],normalize=True))
# print('TouScaleUp',calc_mae2(forecasts['Perfect'],forecasts['TouScaleUp'],normalize=True))

energy_prices = build_price_vectors(load.index,tariff,cfg).energy.values
forecasts['EnergyPricesDn'] = forecasts['Perfect'] - energy_prices * (1.28408091 * maxload)
forecasts['EnergyPricesUp'] = forecasts['Perfect'] + energy_prices * (1.28408091 * maxload)
print('EnergyPricesDn',calc_mae2(forecasts['Perfect'],forecasts['EnergyPricesDn'],normalize=True))
print('EnergyPricesUp',calc_mae2(forecasts['Perfect'],forecasts['EnergyPricesUp'],normalize=True))

demand_prices = build_price_vectors(load.index,tariff,cfg).demand.values
forecasts['DemandPricesDn'] = forecasts['Perfect'] - demand_prices * (0.02796934 * maxload)
forecasts['DemandPricesUp'] = forecasts['Perfect'] + demand_prices * (0.02796934 * maxload)
print('DemandPricesDn',calc_mae2(forecasts['Perfect'],forecasts['DemandPricesDn'],normalize=True))
print('DemandPricesUp',calc_mae2(forecasts['Perfect'],forecasts['DemandPricesUp'],normalize=True))

std=forecasts['Perfect'].std()
forecasts['Random'] = forecasts['Perfect'] + (0.12571028*maxload)*np.random.normal(0,1,len(forecasts))
print('Random',calc_mae2(forecasts['Perfect'],forecasts['Random'],normalize=True))


#forecasts = forecasts.drop([cfg.load_col,cfg.load_persist_col],axis=1)

#forecast.to_csv('data/load/2019_JPL_v4_fake_forecast.csv')
forecasts.to_csv('data/solar/Jail_load_solar_1min_2021-4-21_2022-4-19_profilized_fake_solar_forecast.csv')

#forecasts = forecasts.columns

Perfect 0.0
ConstantDn 0.10000000000000002
ConstantUp 0.10000000000000002
ScaleLinDn 0.09999999807079149
ScaleLinUp 0.09999999807079149
ScaleQuadDn 0.09999999885478637
ScaleQuadUp 0.09999999885478637
EnergyPricesDn 0.0999999996349315
EnergyPricesUp 0.0999999996349315
DemandPricesDn 0.0999999690410959
DemandPricesUp 0.09999996904109588
Random 0.10115801917889185


In [18]:
target_nmae = 0.10#*maxload

d = solar[cfg.solar_col]/maxload
r = np.random.normal(0,1,len(forecasts))/maxload

s = np.random.normal(0, 1, 8760)
def forecast(x):
    #return d + x[0]
    #return d + d * x[0]
    #return d + d**2 * x[0]
    #return d + x[0]*tou_levels
    #return d + x[0]*energy_prices
    #return d + x[0]*demand_prices
    return d+r*x[0]*maxload

def obj(x):
    return (np.mean(np.abs(d-forecast(x))) - target_nmae)**2

linear_constraint = LinearConstraint([[1]], [0], [100])

r = minimize(obj,np.array([0.5]),options={'disp':True},constraints=[linear_constraint])

print('Optimal values of x:',r.x)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.1660917669954415e-17
            Iterations: 3
            Function evaluations: 6
            Gradient evaluations: 3
Optimal values of x: [0.12571028]
